# Curso de Web Scraping

<img src="https://yaelmanuel.com/wp-content/uploads/2021/12/platzi-banner-logo-matematicas.png" width="500px">

---

## 0) Dependencias

In [ ]:
import requests
from bs4 import BeautifulSoup

import csv
import json

import time
import random

## 1) Paginación y scraping de múltiples páginas

In [ ]:
base_url = "http://books.toscrape.com/catalogue/category/books_1/page-{}.html"

In [ ]:
product_list = []

# Recorrer las primeras 3 páginas
for page in range(49, 53):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    products = soup.select("article.product_pod")
    
    for product in products:
        title = product.find("h3").find("a")["title"]
        price = product.find("p", class_="price_color").get_text()
        image_rel = product.find("div", class_="image_container").find("img")["src"]
        image_url = "http://books.toscrape.com/" + image_rel.replace("../", "")
        product_list.append({
            "title": title,
            "price": price,
            "image_url": image_url
        })
    
    # Espera breve entre páginas para simular navegación real
    time.sleep(1)
    print(f"Página {page} procesada.")

In [ ]:
with open("resultados/productos_multi.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["title", "price", "image_url"])
    writer.writeheader()
    writer.writerows(product_list)

print(f"Scraping multipágina completado: {len(product_list)} productos guardados en productos_multi.csv")

## 2) Manejando errores y excepciones comunes

In [ ]:
product_list = []

for page in range(47, 53):  # Probar con 6 páginas
    url = base_url.format(page)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lanza error para códigos 400 o 500
        soup = BeautifulSoup(response.text, "html.parser")
        products = soup.select("article.product_pod")
    except requests.RequestException as e:
        print(f"Error en la página {page}: {e}")
        continue  # Sigue con la siguiente iteración

    for product in products:
        try:
            title = product.find("h3").find("a")["title"]
            price = product.find("p", class_="price_color").get_text()
            image_rel = product.find("div", class_="image_container").find("img")["src"]
            image_url = "http://books.toscrape.com/" + image_rel.replace("../", "")
            product_list.append({
                "title": title,
                "price": price,
                "image_url": image_url
            })
        except Exception as ex:
            print("Error extrayendo datos de un producto:", ex)
    time.sleep(1)
    print(f"Página {page} procesada.")

In [ ]:
with open("resultados/productos_con_errores.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["title", "price", "image_url"])
    writer.writeheader()
    writer.writerows(product_list)

print(f"Scraping completado con manejo de errores: {len(product_list)} productos guardados en productos_con_errores.csv")

## 3) Buenas prácticas: headers, tiempos y ética del scraping

**📜 ¿Qué es robots.txt?**

Es un archivo que los sitios web colocan en su raíz (https://sitio.com/robots.txt) para indicar qué **partes del sitio pueden o no ser exploradas por bots**. Aunque no es una "ley" (no lo impide técnicamente), es una norma ética respetarla.

In [ ]:
# Definir una cabecera
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

In [ ]:
base_url = "http://books.toscrape.com/catalogue/category/books_1/page-{}.html"

In [ ]:
product_list = []

for page in range(1, 4):
    url = base_url.format(page)
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        products = soup.select("article.product_pod")
    except requests.RequestException as e:
        print(f"Error en la página {page}: {e}")
        continue

    for product in products:
        try:
            title = product.find("h3").find("a")["title"]
            price = product.find("p", class_="price_color").get_text()
            image_rel = product.find("div", class_="image_container").find("img")["src"]
            image_url = "http://books.toscrape.com/" + image_rel.replace("../", "")
            product_list.append({
                "title": title,
                "price": price,
                "image_url": image_url
            })
        except Exception as ex:
            print("Error extrayendo datos de un producto:", ex)
    
    # Pausa aleatoria para imitar comportamiento humano
    sleep_time = random.uniform(1, 3)
    time.sleep(sleep_time)
    print(f"Página {page} procesada con una pausa de {sleep_time:.2f} segundos.")

## 4) Guardar como CSV y JSON

### **Guardar en CSV**

In [ ]:
with open("resultados/productos_eticos.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["title", "price", "image_url"])
    writer.writeheader()
    writer.writerows(product_list)

print(f"Scraping ético completado: {len(product_list)} productos guardados en productos_eticos.csv")

### **Guardar en JSON**

In [ ]:
with open("resultados/productos_final.json", "w", encoding="utf-8") as jsonfile:
    json.dump(product_list, jsonfile, indent=4, ensure_ascii=False)

print(f"Datos exportados: {len(product_list)} productos en productos_final.json")

### **Guardar en un Excel**

In [ ]:
#!pip install pandas openpyxl

In [ ]:
import pandas as pd

In [ ]:
# Convertir en Excel
df = pd.DataFrame(product_list)

# Guardar como archivo Excel
df.to_excel("resultados/productos_eticos.xlsx", index=False)

print(f"Scraping ético completado: {len(product_list)} productos guardados en productos_eticos.xlsx")

### **Guardar en un Google Form**

In [ ]:
import requests
import time

Ejemplo de la estructura de URL de un formulario: 
<br>https://docs.google.com/forms/d/e/1FAIpQLSfwOqkiwDrLLMyi8-YllxysuERDhsaXsu6oo1398y5b4Vl85A/viewform?usp=pp_url&entry.1649230915=Pinocho&entry.80026608=1500&entry.1487693205=www.pinocho.com

In [ ]:
# URL del formulario
url = "https://docs.google.com/forms/d/e/xxx/formResponse"

# Encabezados para evitar error 401
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    "Referer": "https://docs.google.com/forms/d/e/xxx/viewform"
}

# Recorrer y enviar cada producto
for i, product in enumerate(product_list[0:5], start=1):
    payload = {
        "entry.xxx": product["title"],        # campo 1: título
        "entry.xxx": product["price"],       # campo 2: precio
        "entry.xxx": product["image_url"]    # campo 3: imagen
    }

    response = requests.post(url, data=payload, headers=headers)

    if response.status_code == 200:
        print(f"✅ Producto {i} enviado: {product['title']}")
    else:
        print(f"❌ Error al enviar producto {i} - Código: {response.status_code}")
    
    time.sleep(1)  # espera 1 segundo entre envíos para evitar bloqueos

✅ Producto 1 enviado: A Light in the Attic
✅ Producto 2 enviado: Tipping the Velvet
✅ Producto 3 enviado: Soumission
✅ Producto 4 enviado: Sharp Objects
✅ Producto 5 enviado: Sapiens: A Brief History of Humankind
